In [1]:
import ast
import requests
import json
import pandas as pd
import re
from urllib.request import Request, urlopen
import urllib.request
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import time
import numpy as np
import random
import re

In [2]:
# Activation of Chrome Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#mettre le bon chemin 
driver = webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

#recupérer la page web
driver.get("https://www.persee.fr/search?ta=article&q=fuite+des+cerveaux+europe")

/home/biglifa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [3]:
#récupérerer le nombre d'articles
nb_resulats = driver.find_elements_by_class_name("result-count")[0].text
nb_resulats = int(nb_resulats[0:6])

data = pd.DataFrame(index=list(np.arange(nb_resulats)),
    columns=['Title','Context','Year_publication','Collection','Type','Authors','Document_issue','Page_issue'])

data.head()

,Title,Context,Year_publication,Collection,Type,Authors,Document_issue,Page_issue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# fonction pause
def pause():
    time_break = random.randint(1, 2)
    return time.sleep(time_break)

#Je vais recuperer seulement les 100 premiere articles
#Pour recuperer tout, remplacer 100 par nb_resulats 
N=0
while N<100:
    
    # scroller jusqu'en bas de page
    driver.execute_script ("window.scrollTo (0, document.body.scrollHeight);")
    docs=driver.find_elements_by_class_name("doc-result")
    N=len(docs)
    pause()

In [5]:
i=0
for doc in docs: 
    
    #Title
    try:
        title=doc.find_elements_by_class_name("title")[0].text
    except NoSuchElementException:
        title=''
        pass
    
    #author
    try:
        author=[a.text for a in doc.find_elements_by_class_name("contributor")]
        author=str(author)[1:-1]
    except NoSuchElementException:
        author=''
        pass
            
    
    #Collection
    try:
        collection=doc.find_element_by_class_name("collection").text 
    except NoSuchElementException:
        collection=''
        pass
    
    #Type
    try:
        m=re.search(r"\[\w+([ ',-]\w+)*\]",doc.find_elements_by_css_selector("div")[0].text)
        typee=str(m.group(0))[1:-1]
    except:
        typee=''
        pass
    
    #Date
    try:
        date=doc.find_element_by_class_name("documentYear").text[5:]
    except NoSuchElementException:
        date=''
        pass
    
    
    #Document issue
    try:
        docIssu=doc.find_element_by_class_name("documentIssue").text
    except NoSuchElementException:
        docIssu=''
        pass
    
    
    #Page issue
    try :
        page=doc.find_element_by_class_name("documentPageRange").text[3:]
    except:
        page=''
        pass

    #context
    try:
        context=doc.find_element_by_class_name("searchContext").text
    except NoSuchElementException:
        context=''
        pass
    
    data.loc[i,:]=[title,context,date,collection,typee,author,docIssu,page]
    
    # scroll doucement
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center', inline: 'nearest'});", doc)
        
    i+=1
data.head()

,Title,Context,Year_publication,Collection,Type,Authors,Document_issue,Page_issue
0,La fuite des cerveaux : le cas de l'ex-URSS,La fuite des cerveaux : le cas de l'ex-URSS......,1992,Hommes et Migrations,article,"'Andrei Kortunov', 'Rachel Bouyssou [Traducteur]'",1155,22-25
1,Que nous devons-nous les uns aux autres ? Le c...,La fuite des cerveaux est-elle un phénomène in...,2008,Revue Philosophique de Louvain,article,'Stéphane Chauvier',106-4,771-796
2,"Croissance, redistribution et inégalités dans ...","Croissance, redistribution et inégalités dans ...",1999,Revue économique,article,"'Frédéric Docquier', 'Hillel Rapoport'",50-3,499-510
3,Au-delà de la « fuite des cerveaux » : la mobi...,Au-delà de la « fuite des cerveaux » : la mobi...,1997,Revue européenne des migrations internationales,article,"'Félicitas Hillmann', 'Hedwig Rudolph'",13-1,71-94
4,Chorafas Dimitri N. — La fuite des cerveaux. L...,Chorafas Dimitri N. — La fuite des cerveaux. L...,1970,Population,compte-rendu,'J-N B',25-5,1131-1132


In [6]:
#Tranformer en dataframe
data.to_csv(index = True, sep=',', path_or_buf='../data/donnees.csv')